In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk
# % matplotlib inline
#plt.rcParams['animation.html'] = 'jshtml'
from IPython.display import HTML
import cv2

import os
import sys
from functools import reduce
sys.path.append("/home/ful7rng/projects/transition/propagation")
sys.path.append("/home/ful7rng/projects/transition/data_generator/multi_agent_planning/scripts")
from utils import pickle_load, get_npy, get_file
from utils.occ_map_utils import load_map, show_map, plot_occ_map, plot_trajectories, free_space
from utils.scene_utils import display_scenes_trajs, animate_scenes
from pyastar import astar_path
from data_generator.astar_ped_sim.astar_traj_generator import AStarGrid


In [2]:
def get_cost_map(map_arr, blur1_cost=5, blur2_cost=1, cc=20, bw1=5, bw2=11):
    # calculate cost map: make walls expensive and blur them
    collision_cost = cc

    blur1_width = bw1
    blurred_map1 = cv2.GaussianBlur(map_arr.astype(float),
                                   (blur1_width, blur1_width), 0)
    blur2_width = bw2
    blurred_map2 = cv2.GaussianBlur(map_arr.astype(float),
                                   (blur2_width, blur2_width), 0)

    cost_map = collision_cost * map_arr.astype(float) + \
               blur1_cost * blurred_map1 + blur2_cost * blurred_map2
    return cost_map

In [3]:
from data_generator.astar_ped_sim.astar_traj_generator import _sample_trajectories

def generate_trajs(map_arr, cost_map, diagonal=False):
    verbose = True
    num_trajectories = 5
    min_dist = 10
    max_dist = 40
    trajectories = []
    start_locations = list(zip(*np.where(np.logical_not(map_arr))))
    start_locations = [np.array(loc) for loc in start_locations]
    num_locations = len(start_locations)
    for idx, loc in enumerate(start_locations):
        if verbose and idx % 1 == 0:
            print('sampling trajectories for possible start locations: {}/{}'.format(
                idx + 1, num_locations))

            trajectories_ = _sample_trajectories(loc, map_arr, cost_map,
                                                 num_trajectories,
                                                 min_dist, max_dist, None, diagonal)
            if trajectories_ is not None:
                trajectories += trajectories_
    return trajectories

In [114]:
start = [23, 19]
end = [48, 13]

In [132]:
map_ = np.load("/home/ful7rng/projects/transition/data_generator/map_arr.npy")
cost_map = get_cost_map(map_)
walls = set([tuple(loc) for loc in np.array(np.where(map_)).T.tolist()])

In [116]:
ax = show_map(cost_map, resolution=1, cmap='Greens', zorder=0)
plt.colorbar(ax)

In [117]:
#show_map(map_, resolution=1)
plt.plot([start[0], end[0]], [start[1], end[1]])

In [130]:
a_star_grid = AStarGrid(cost_map, diagonal=True)
path, cost = a_star_grid.astar(tuple(start), tuple(end), max_cost=1999)
traj_traces = [loc for loc in path]

In [131]:
traj_traces

[(23, 19),
 (24, 20),
 (25, 20),
 (26, 20),
 (27, 19),
 (28, 18),
 (29, 17),
 (30, 17),
 (31, 17),
 (32, 17),
 (33, 17),
 (34, 17),
 (35, 17),
 (36, 17),
 (37, 17),
 (38, 17),
 (39, 17),
 (40, 17),
 (41, 17),
 (42, 17),
 (43, 17),
 (44, 17),
 (45, 16),
 (46, 15),
 (47, 14),
 (48, 13)]

In [125]:
traj_traces_1

['[23 19]',
 '[24 20]',
 '[25 20]',
 '[26 20]',
 '[27 19]',
 '[28 18]',
 '[29 17]',
 '[30 17]',
 '[31 17]',
 '[32 17]',
 '[33 17]',
 '[34 17]',
 '[35 17]',
 '[36 17]',
 '[37 17]',
 '[38 17]',
 '[39 17]',
 '[40 17]',
 '[41 17]',
 '[42 17]',
 '[43 17]',
 '[44 17]',
 '[45 16]',
 '[46 15]',
 '[47 14]',
 '[48 13]']

In [133]:
path_1, values = astar_path(cost_map.astype('float32'), start, end, 1, True)
path_1 = np.insert(path_1, 0, end, axis=0)[::-1]
traj_traces_1 = [tuple(loc) for loc in path_1]

In [134]:
walls.intersection(traj_traces)

{(23, 19), (47, 14), (48, 13)}

In [135]:
walls.intersection(traj_traces_1)

{(23, 19), (47, 14), (48, 13)}

In [136]:
assert traj_traces == traj_traces_1

In [123]:
show_map(map_, resolution=1)
xs = map(lambda coor: coor[0], path)
ys = map(lambda coor: coor[1], path)
plt.plot(xs, ys)

In [124]:
show_map(map_, resolution=1)
plt.plot(path_1[:, 0], path_1[:, 1])

In [5]:
test_map = np.random.randn(50, 60)
start = [10, 20]
end = [30, 40]
astar_path(test_map.astype('float32'), start, end, 1, False)

[array([[30, 41],
        [29, 41],
        [28, 41],
        [28, 40],
        [28, 39],
        [28, 38],
        [27, 38],
        [27, 37],
        [26, 37],
        [26, 36],
        [25, 36],
        [25, 35],
        [25, 34],
        [25, 33],
        [25, 32],
        [24, 32],
        [24, 31],
        [23, 31],
        [22, 31],
        [21, 31],
        [21, 30],
        [21, 29],
        [21, 28],
        [20, 28],
        [19, 28],
        [18, 28],
        [17, 28],
        [17, 27],
        [17, 26],
        [16, 26],
        [15, 26],
        [15, 25],
        [15, 24],
        [14, 24],
        [14, 23],
        [13, 23],
        [13, 22],
        [12, 22],
        [12, 21],
        [11, 21],
        [11, 20],
        [10, 20]]), array([[ inf,  inf,  inf, ...,  inf,  inf,  inf],
        [ inf,  inf,  inf, ...,  inf,  inf,  inf],
        [ inf,  inf,  inf, ...,  inf,  inf,  inf],
        ..., 
        [ inf,  inf,  inf, ...,  inf,  inf,  inf],
        [ inf,  inf,  inf

### try different cost maps

In [4]:
from data_generator.human_mcm import Grid_HMM

def get_trans_probs(map_, trajs, diagonal=False):
    mcm = Grid_HMM(np.array(map_.shape).astype(int))
    for idx, trajectory in enumerate(trajs):
        if idx % 100 == 0:
            print("Add transitions of {}/{} trajectory to the mcm.".format(idx+1, len(trajs)))
        #trajectory = np.round((trajectory / res) - 0.5).astype(int)
        #print(str(trajectory))
        for t in range(trajectory.shape[0] - 2):
            # give first cell seven prior locations
            # and add them to transitions
            if t == 0:
                first_cell = trajectory[0, :]
                second_cell = trajectory[1, :]
                velocities =  [[0, 1], [1, 0], [0, -1], [-1, 0],
                          [1, 1], [1, -1], [-1, 1], [-1, -1]]
                if not diagonal:
                    velocities = velocities[:4]
                prior_cells = [[first_cell[0]+i, first_cell[1]+j]
                               for i, j in velocities]
                for cell in prior_cells:
                    if cell[0] != second_cell[0] or cell[1] != second_cell[1]:
                        mcm.add_transition(cell, first_cell, second_cell)

            from_ = trajectory[t, :]
            current = trajectory[t + 1, :]
            to = trajectory[t + 2, :]
            mcm.add_transition(from_, current, to)

    transition_probs = mcm.get_transition_probs()
    return transition_probs

In [5]:
from utils.plot_utils import plot_4d_tensor

def onclick(event, bofum, fig, fig_1, fig_2, plot):
    ix, iy = event.xdata, event.ydata
    coords = np.floor(np.array([ix, iy]) / bofum.map_res).astype(int)
    print(coords)
    clicked = np.zeros_like(bofum.map)
    x, y = coords[0], coords[1]
    clicked[x, y] = 1
    plot.set_axes_data("occupancy_axes", clicked)
    fig.canvas.draw()
    
    fig_1.clear()
    fig_2.clear()
    condi_probs = bofum.conditional_probs
    prob = condi_probs[x, y]
    print(np.sum(prob, axis=(2, 3)))
    print(np.sum(prob))
    plot_4d_tensor(condi_probs[x, y], fig=fig_1)
    plot_4d_tensor(bofum.kernels[x, y], fig=fig_2)
    fig_1.canvas.draw()
    fig_2.canvas.draw()

In [19]:

def show_map_and_probs(map_, transition_probs_1):

    kwargs = dict(omega=0.05,
                  noise_var=0.5,
                  extent=7,
                  verbose=False)

    #model_name = '23_ALL_MAPS_EIGHT_DIRECTIONS'
    model_name = '20_ALL_MAPS_W_MASK'

    bofum_2 = conditionalBOFUM(map_, model_name,
                                       name = 'BOFUM with n. o. as acceleration',
                                       simulated_data=True,
                                       with_reachability=True,
                                       acceleration_interpretation=True,
                                       condi_probs=None,
                                       force_predict=False,
                                       **kwargs)

    bofum_2.conditional_probs = transition_probs_1
    distances, trajs = bofum_2.initialize(num_targets=5, num_steps=20, diagonal=False)
    bofum_2.show_trajectories()

    from propagation.animation import Plot
    fig_6 = plt.figure(figsize=(5, 5))
    fig_7 = plt.figure(figsize=(6, 5))
    fig_8 = plt.figure(figsize=(6, 5))

    map_axes_2 = fig_6.add_subplot(111)
    plot_2 = Plot(map_axes_2, bofum_2.map, bofum_2.map_res)
    fig_6.canvas.mpl_connect('button_press_event', lambda event: onclick(event, bofum_2, fig_6, fig_7, fig_8, plot_2))
    return bofum_2

In [7]:
from propagation.bofum import conditionalBOFUM, naiveBOFUM
from data_loader import get_map_crop

config_path = '/local/home/ful7rng/projects/transition/config_validation.py'
maps, outputs = get_map_crop(config_path, num=1, dataset='train', condi_probs=True)
map_ = maps[0]
output = outputs[0]

(164, 32, 32)


In [8]:
cost_map = get_cost_map(map_)
trajs = generate_trajs(map_, cost_map)

sampling trajectories for possible start locations: 1/420
sampling trajectories for possible start locations: 2/420
sampling trajectories for possible start locations: 3/420
sampling trajectories for possible start locations: 4/420
sampling trajectories for possible start locations: 5/420
sampling trajectories for possible start locations: 6/420
sampling trajectories for possible start locations: 7/420
sampling trajectories for possible start locations: 8/420
sampling trajectories for possible start locations: 9/420
sampling trajectories for possible start locations: 10/420
sampling trajectories for possible start locations: 11/420
sampling trajectories for possible start locations: 12/420
sampling trajectories for possible start locations: 13/420
sampling trajectories for possible start locations: 14/420
sampling trajectories for possible start locations: 15/420
sampling trajectories for possible start locations: 16/420
sampling trajectories for possible start locations: 17/420
sampli

sampling trajectories for possible start locations: 141/420
sampling trajectories for possible start locations: 142/420
sampling trajectories for possible start locations: 143/420
sampling trajectories for possible start locations: 144/420
sampling trajectories for possible start locations: 145/420
sampling trajectories for possible start locations: 146/420
sampling trajectories for possible start locations: 147/420
sampling trajectories for possible start locations: 148/420
sampling trajectories for possible start locations: 149/420
sampling trajectories for possible start locations: 150/420
sampling trajectories for possible start locations: 151/420
sampling trajectories for possible start locations: 152/420
sampling trajectories for possible start locations: 153/420
sampling trajectories for possible start locations: 154/420
sampling trajectories for possible start locations: 155/420
sampling trajectories for possible start locations: 156/420
sampling trajectories for possible start

sampling trajectories for possible start locations: 289/420
sampling trajectories for possible start locations: 290/420
sampling trajectories for possible start locations: 291/420
sampling trajectories for possible start locations: 292/420
sampling trajectories for possible start locations: 293/420
sampling trajectories for possible start locations: 294/420
sampling trajectories for possible start locations: 295/420
sampling trajectories for possible start locations: 296/420
sampling trajectories for possible start locations: 297/420
sampling trajectories for possible start locations: 298/420
sampling trajectories for possible start locations: 299/420
sampling trajectories for possible start locations: 300/420
sampling trajectories for possible start locations: 301/420
sampling trajectories for possible start locations: 302/420
sampling trajectories for possible start locations: 303/420
sampling trajectories for possible start locations: 304/420
sampling trajectories for possible start

In [9]:
len(trajs)

2100

In [10]:
trans_probs = get_trans_probs(map_, trajs)

Add transitions of 1/2100 trajectory to the mcm.
Add transitions of 101/2100 trajectory to the mcm.
Add transitions of 201/2100 trajectory to the mcm.
Add transitions of 301/2100 trajectory to the mcm.
Add transitions of 401/2100 trajectory to the mcm.
Add transitions of 501/2100 trajectory to the mcm.
Add transitions of 601/2100 trajectory to the mcm.
Add transitions of 701/2100 trajectory to the mcm.
Add transitions of 801/2100 trajectory to the mcm.
Add transitions of 901/2100 trajectory to the mcm.
Add transitions of 1001/2100 trajectory to the mcm.
Add transitions of 1101/2100 trajectory to the mcm.
Add transitions of 1201/2100 trajectory to the mcm.
Add transitions of 1301/2100 trajectory to the mcm.
Add transitions of 1401/2100 trajectory to the mcm.
Add transitions of 1501/2100 trajectory to the mcm.
Add transitions of 1601/2100 trajectory to the mcm.
Add transitions of 1701/2100 trajectory to the mcm.
Add transitions of 1801/2100 trajectory to the mcm.
Add transitions of 1901/

In [20]:
bufum = show_map_and_probs(map_, trans_probs)

sampling trajectories: 1/5
time out
sampling trajectories: 1/5


In [16]:
bofum.show_trajectories()

NameError: name 'bofum' is not defined